In [8]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
import numpy as np
from sklearn import preprocessing
from tensorflow.keras.datasets import mnist

# Define the One-hot Encoder
ohe = preprocessing.OneHotEncoder()

# Load MNIST data
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape data
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Fit and transform training data
ohe.fit(y_train)
transformed_train = ohe.transform(y_train).toarray()

# Fit and transform testing data
ohe.fit(y_test)
transformed_test = ohe.transform(y_test).toarray()

x_train = x_train[0:50000, :]
#y_train = transformed_train[0:50000,:]
transformed_train = transformed_train[0:50000,:]

In [10]:
print('X_train: ' + str(x_train.shape))
print('Y_train: ' + str(transformed_train.shape))
print('X_test:  '  + str(x_test.shape))
print('Y_test:  '  + str(transformed_test.shape))

X_train: (50000, 28, 28)
Y_train: (50000, 10)
X_test:  (10000, 28, 28)
Y_test:  (10000, 10)


In [11]:
train_X_mod = x_train.reshape(50000, 784)
train_y_mod = transformed_train
test_X_mod = x_test.reshape(10000, 784)
test_y_mod = transformed_test

In [12]:
size_input = 784
size_hidden1 = 512
size_hidden2 = 256
size_output = 10
number_of_train_examples = 50000
number_of_test_examples = 10000

In [13]:
import tensorflow as tf
X_train = tf.keras.utils.normalize(train_X_mod, axis=1)
y_train = train_y_mod
X_test = test_X_mod
y_test = test_y_mod 

In [14]:
import os
import numpy as np
import time

np.random.seed(43)
tf.random.set_seed(43)
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

In [18]:
class MLP(object):
  def __init__(self, size_input, size_hidden1, size_hidden2, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of hidden layer 1
    size_hidden2: int, size of hodden layer 2
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_output, self.device =\
    size_input, size_hidden1, size_hidden2, size_output, device
    
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1],stddev=0.1))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden1]))
    # Initialize weights between input layer and hidden layer
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2],stddev=0.1))
    # Initialize biases for hidden layer
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden2]))
     # Initialize weights between hidden layer and output layer
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_output],stddev=0.1))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    #return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)
    #print(y_true_tf)
    #print(y_pred_tf)
    cce = tf.keras.losses.CategoricalCrossentropy()
    loss_val = cce(y_true_tf, y_pred_tf)
    #regularizer = tf.nn.l2_loss(self.W1)+tf.nn.l2_loss(self.W2)
    #loss_val = tf.reduce_mean(loss_val + 0.01 * regularizer)
    
    #print(loss_val)
    return loss_val

  def accuracy(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    #y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)
    print(y_true_tf)
    print(y_pred_tf)
    ## CALCULATING COST AND ACCURACY
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_preds, labels=y))
    #optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    #print(correct_pred)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    #print(ac)
    return accuracy
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    #y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)    
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    #print(correct_pred)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    #print(accuracy)
    return accuracy
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    #Remember to normalize your dataset before moving forward
    # Compute values in hidden layer1
    what1 = tf.matmul(X_tf, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)
    #hhat1_1 = tf.nn.dropout(hhat1, keep_prob)
    # Compute values in hidden layer2
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)
    # Compute output
    output = tf.matmul(hhat2, self.W3) + self.b3
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    output = tf.nn.softmax(output)
    return output

In [21]:
# Set number of epochs
NUM_EPOCHS = 10

In [22]:
# Initialize model using CPU
mlp_on_cpu = MLP(size_input, size_hidden1, size_hidden2, size_output, device='cpu')

# Array to store accuracy and loss
loss_with_epoch = []
acc_with_epoch = []

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  ac = 0
  count = 0
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(20)
  for inputs, outputs in train_ds:
    preds = mlp_on_cpu.forward(inputs)
    loss_total = loss_total + mlp_on_cpu.loss(preds, outputs)
    lt = lt + mlp_on_cpu.loss(preds, outputs)
    mlp_on_cpu.backward(inputs, outputs)
    ac = ac+mlp_on_cpu.accuracy(preds, outputs)
    #ac = mlp_on_cpu.accuracy(preds, outputs)
    count += 1
  print('Number of Epoch = {} - Average celoss:= {}- Acc:= {} '.format(epoch + 1, np.sum(loss_total) / X_train.shape[0], ac/count))
  loss_with_epoch.append(np.sum(loss_total) / X_train.shape[0])
  acc_with_epoch.append(ac/count)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs

Number of Epoch = 1 - Average celoss:= 0.1051168359375- Acc:= 0.479139506816864 
Number of Epoch = 2 - Average celoss:= 0.075668759765625- Acc:= 0.7422204613685608 
Number of Epoch = 3 - Average celoss:= 0.054981630859375- Acc:= 0.7971610426902771 
Number of Epoch = 4 - Average celoss:= 0.0419624072265625- Acc:= 0.8236796855926514 
Number of Epoch = 5 - Average celoss:= 0.03424661376953125- Acc:= 0.8416399359703064 
Number of Epoch = 6 - Average celoss:= 0.02949181396484375- Acc:= 0.8526199460029602 
Number of Epoch = 7 - Average celoss:= 0.02637247314453125- Acc:= 0.8610800504684448 
Number of Epoch = 8 - Average celoss:= 0.02421305419921875- Acc:= 0.868079423904419 
Number of Epoch = 9 - Average celoss:= 0.0226669775390625- Acc:= 0.8734791874885559 
Number of Epoch = 10 - Average celoss:= 0.02151366943359375- Acc:= 0.8771976828575134 

Total time taken (in seconds): 777.17


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x = [i for i in range(1,21)]
line1, = plt.plot(x, acc_with_epoch, label='accr')
line2, = plt.plot(x, loss_with_epoch, label='ce loss')
#plt.plot(x, acc_with_epoch, label='accr')
plt.legend(handles=[line1, line2], loc='best')
plt.grid(b=True, color='aqua', alpha=0.6, linestyle='dashdot')
plt.show()

In [ ]:
# Visualizing that pred output is same as actual output for testing
def print_val(x,y):
    for i in range(5):
        print(x[i])
        print(y[i])
        
print_val(preds, outputs)